In [ ]:
! git clone https://github.com/nikhilroxtomar/FANet.git

Cloning into 'FANet'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 77 (delta 30), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (77/77), 555.99 KiB | 2.87 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
! wget https://datasets.simula.no/downloads/kvasir-seg.zip


--2024-04-10 19:17:28--  https://datasets.simula.no/downloads/kvasir-seg.zip
Resolving datasets.simula.no (datasets.simula.no)... 128.39.36.14
Connecting to datasets.simula.no (datasets.simula.no)|128.39.36.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46227172 (44M) [application/zip]
Saving to: ‘kvasir-seg.zip’

kvasir-seg.zip      100%[===================>]  44.08M  13.2MB/s    in 3.5s    

2024-04-10 19:17:33 (12.6 MB/s) - ‘kvasir-seg.zip’ saved [46227172/46227172]



In [ ]:
! unzip kvasir-seg.zip

In [ ]:
import os
import random

segfilepath=r'/content/Kvasir-SEG/images'
saveBasePath=r"/content/Kvasir-SEG"

trainval_percent=1
# 9:1
train_percent=0.9

temp_seg = os.listdir(segfilepath)
total_seg = []
for seg in temp_seg:
    if seg.endswith(".jpg"):
        total_seg.append(seg)

num=len(total_seg)
list=range(num)
tv=int(num*trainval_percent)
tr=int(tv*train_percent)
trainval= random.sample(list,tv)
train=random.sample(trainval,tr)

print("train and val size",tv)
print("traub suze",tr)
ftrainval = open(os.path.join(saveBasePath,'trainval.txt'), 'w')
ftest = open(os.path.join(saveBasePath,'test.txt'), 'w')
ftrain = open(os.path.join(saveBasePath,'train.txt'), 'w')
fval = open(os.path.join(saveBasePath,'val.txt'), 'w')

for i  in list:
    name=total_seg[i][:-4]+'\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()

train and val size 1000
traub suze 900


In [ ]:
! python FANet/train.py

Log file exists
2024-04-10 19:18:23.667883
Traceback (most recent call last):
  File "/content/FANet/train.py", line 174, in <module>
    (train_x, train_y), (valid_x, valid_y) = load_data(path)
  File "/content/FANet/utils.py", line 37, in load_data
    train_x, train_y = load_names(path, train_names_path)
  File "/content/FANet/utils.py", line 28, in load_names
    f = open(file_path, "r")
FileNotFoundError: [Errno 2] No such file or directory: '../Kvasir-SEG/train.txt'


In [ ]:
! python FANet/test.py

python3: can't open file '/content/FANet/test.py': [Errno 2] No such file or directory


Water Extracting

In [ ]:
from google.colab import userdata
import os

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

In [ ]:
!kaggle datasets download -d waterkingest/water-bodies --force

 99% 1.55G/1.56G [00:22<00:00, 155MB/s]
100% 1.56G/1.56G [00:22<00:00, 74.1MB/s]


In [ ]:
! unzip water-bodies.zip

In [ ]:
import os
import random

segfilepath=r'/content/VOCdevkit/images'
saveBasePath=r"/content/VOCdevkit"

trainval_percent=1
# 9:1
train_percent=0.9

temp_seg = os.listdir(segfilepath)
total_seg = []
for seg in temp_seg:
    if seg.endswith(".png"):
        total_seg.append(seg)

num=len(total_seg)
list=range(num)
tv=int(num*trainval_percent)
tr=int(tv*train_percent)
trainval= random.sample(list,tv)
train=random.sample(trainval,tr)

print("train and val size",tv)
print("traub suze",tr)
ftrainval = open(os.path.join(saveBasePath,'trainval.txt'), 'w')
ftest = open(os.path.join(saveBasePath,'test.txt'), 'w')
ftrain = open(os.path.join(saveBasePath,'train.txt'), 'w')
fval = open(os.path.join(saveBasePath,'val.txt'), 'w')

for i  in list:
    name=total_seg[i][:-4]+'\n'
    if i in trainval:
        ftrainval.write(name)
        if i in train:
            ftrain.write(name)
        else:
            fval.write(name)
    else:
        ftest.write(name)

ftrainval.close()
ftrain.close()
fval.close()
ftest.close()

train and val size 7424
traub suze 6681


In [ ]:
!cp -r /content/VOCdevkit/VOC2007/. /content/VOCdevkit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import time
import datetime
import random
import numpy as np
from glob import glob
import albumentations as A
import cv2
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from utils import (
    seeding, shuffling, create_dir, init_mask,
    epoch_time, rle_encode, rle_decode, print_and_save, load_data
    )
from model import FANet
from loss import DiceBCELoss

class DATASET(Dataset):
    def __init__(self, images_path, masks_path, size, transform=None):
        super().__init__()

        self.images_path = images_path
        self.masks_path = masks_path
        self.transform = transform
        self.n_samples = len(images_path)

    def __getitem__(self, index):
        """ Image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            # print(augmentations)
            image = augmentations["image"]
            mask = augmentations["mask"]

        image = cv2.resize(image, size)
        image = np.transpose(image, (2, 0, 1))
        image = image/255.0
        image = image.astype(np.float32)


        mask = cv2.resize(mask, size)
        mask = np.expand_dims(mask, axis=0)
        mask = mask/255.0
        mask = mask.astype(np.float32)

        return image, mask

    def __len__(self):
        return self.n_samples

def train(model, loader, mask, optimizer, loss_fn, device):
    epoch_loss = 0
    return_mask = []

    model.train()
    i=0
    # for i, (x, y) in enumerate(loader):
    for temp1 in tqdm(loader,total=len(loader)):
        x,y=temp1
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        b, c, h, w  = y.shape
        m = []
        for edata in mask[i*b : i*b+b]:
            edata = " ".join(str(d) for d in edata)
            edata = str(edata)
            edata = rle_decode(edata, size)
            edata = np.expand_dims(edata, axis=0)
            m.append(edata)
        i+=1
        m = np.array(m, dtype=np.int32)
        m = np.transpose(m, (0, 1, 3, 2))
        m = torch.from_numpy(m)
        m = m.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        y_pred = model([x, m])
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            y_pred = torch.sigmoid(y_pred)
            y_pred = y_pred.cpu().numpy()

            for py in y_pred:
                py = np.squeeze(py, axis=0)
                py = py > 0.5
                py = np.array(py, dtype=np.uint8)
                py = rle_encode(py)
                return_mask.append(py)

        epoch_loss += loss.item()

    epoch_loss = epoch_loss/len(loader)
    return epoch_loss, return_mask

def evaluate(model, loader, mask, loss_fn, device):
    epoch_loss = 0
    return_mask = []

    model.eval()
    with torch.no_grad():
        i=0
        # for i, (x, y) in enumerate(loader):
        for temp1 in tqdm(loader,total=len(loader)):
            x,y=temp1

            x = x.to(device)
            y = y.to(device)

            b, c, h, w  = y.shape
            m = []
            for edata in mask[i*b : i*b+b]:
                edata = " ".join(str(d) for d in edata)
                edata = str(edata)
                edata = rle_decode(edata, size)
                edata = np.expand_dims(edata, axis=0)
                m.append(edata)
            i+=1
            m = np.array(m)
            m = np.transpose(m, (0, 1, 3, 2))
            m = torch.from_numpy(m)
            m = m.to(device, dtype=torch.float32)


            y_pred = model([x, m])
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

            y_pred = torch.sigmoid(y_pred)
            y_pred = y_pred.cpu().numpy()

            for py in y_pred:
                py = np.squeeze(py, axis=0)
                py = py > 0.5
                py = np.array(py, dtype=np.uint8)
                py = rle_encode(py)
                return_mask.append(py)

    epoch_loss = epoch_loss/len(loader)
    return epoch_loss, return_mask

if __name__ == "__main__":
    """ Seeding """
    seeding(42)

    """ Directories """
    create_dir("files")

    """ Training logfile """
    train_log_path = "files/train_log.txt"
    if os.path.exists(train_log_path):
        print("Log file exists")
    else:
        train_log = open("files/train_log.txt", "w")
        train_log.write("\n")
        train_log.close()

    """ Record Date & Time """
    datetime_object = str(datetime.datetime.now())
    print_and_save(train_log_path, datetime_object)

    """ Hyperparameters """
    size = (512, 512)
    batch_size = 4
    num_epochs = 10
    lr = 1e-4
    checkpoint_path = "files/checkpoint.pth"

    """ Dataset """
    path = "/content/VOCdevkit"
    (train_x, train_y), (valid_x, valid_y) = load_data(path)
    print(train_x)
    train_x, train_y = shuffling(train_x, train_y)

    data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}\n"
    print_and_save(train_log_path, data_str)

    """ Data augmentation: Transforms """
    transform =  A.Compose([
        A.Rotate(limit=35, p=0.3),
        A.HorizontalFlip(p=0.3),
        A.VerticalFlip(p=0.3),
        A.CoarseDropout(p=0.3, max_holes=10, max_height=32, max_width=32)
    ])

    """ Dataset and loader """
    train_dataset = DATASET(train_x, train_y, size, transform=transform)
    valid_dataset = DATASET(valid_x, valid_y, size, transform=None)

    data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}\n"
    print_and_save(train_log_path, data_str)

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    valid_loader = DataLoader(
        dataset=valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    """ Model """
    device = torch.device('cuda')
    model = FANet()
    # model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
    loss_fn = DiceBCELoss()
    loss_name = "BCE Dice Loss"

    data_str = f"Hyperparameters:\nImage Size: {size}\nBatch Size: {batch_size}\nLR: {lr}\nEpochs: {num_epochs}\n"
    data_str += f"Optimizer: Adam\nLoss: {loss_name}\n"
    print_and_save(train_log_path, data_str)

    """ Training the model. """
    best_valid_loss = float('inf')
    train_mask = init_mask(train_x, size)
    valid_mask = init_mask(valid_x, size)

    for epoch in range(num_epochs):
        start_time = time.time()

        train_loss, return_train_mask = train(model, train_loader, train_mask, optimizer, loss_fn, device)
        valid_loss, return_valid_mask = evaluate(model, valid_loader, valid_mask, loss_fn, device)
        scheduler.step(valid_loss)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            data_str = f"Saving checkpoint: {checkpoint_path}"
            print_and_save(train_log_path, data_str)
            torch.save(model.state_dict(), checkpoint_path)

            train_mask = return_train_mask
            valid_mask = return_valid_mask

        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
        data_str += f'\tTrain Loss: {train_loss:.3f}\n'
        data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
        print_and_save(train_log_path, data_str)


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Log file exists
2024-04-10 20:56:48.393810
['/content/VOCdevkit/images/H48F016018_clip2_49.png', '/content/VOCdevkit/images/H48F019018_clip4_57.png', '/content/VOCdevkit/images/H48F015015_clip9_64.png', '/content/VOCdevkit/images/H48F016017_clip1_56.png', '/content/VOCdevkit/images/H48F018018_clip1_39.png', '/content/VOCdevkit/images/H48F015015_clip9_27.png', '/content/VOCdevkit/images/H48F017018_clip9_26.png', '/content/VOCdevkit/images/H48F014016_clip1_49.png', '/content/VOCdevkit/images/H48F017017_clip1_8.png', '/content/VOCdevkit/images/H48F019017_clip1_46 (2).png', '/content/VOCdevkit/images/H48F016017_clip4_16.png', '/content/VOCdevkit/images/H48F015018_clip3_52.png', '/content/VOCdevkit/images/H48F018017_clip7_28.png', '/content/VOCdevkit/images/H48F018017_clip1_22 (2).png', '/content/VOCdevkit/images/dqh3_1_34.png', '/content/VOCdevkit/images/dqh6_5_50.png', '/content/VOCdevkit/images/H48F016017_clip1_38.png', '/content/VOCdevkit/images/H48F018018_clip2_19.png', '/content/VOCde

  0%|          | 0/1671 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|██████████| 186/186 [01:14<00:00,  2.50it/s]


Saving checkpoint: files/checkpoint.pth
Epoch: 01 | Epoch Time: 28m 18s
	Train Loss: 0.595
	 Val. Loss: 0.468



100%|██████████| 186/186 [01:09<00:00,  2.66it/s]


Saving checkpoint: files/checkpoint.pth
Epoch: 02 | Epoch Time: 27m 42s
	Train Loss: 0.435
	 Val. Loss: 0.401



  3%|▎         | 46/1671 [00:45<25:40,  1.05it/s]